In [1]:
import fundamentalanalysis as fa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
from dateutil.relativedelta import relativedelta


%matplotlib inline  

In [2]:
# api_key = '00ef9804fcde0edd93b1b4821ee2f06a' #job.chap@icloud.com
# api_key = 'c81352430e2fe3c941faf0814227562b' #jobs.chaps@gmail.com
api_key = 'a765d11740cccfb61177da0ad8699d1e' #job.chap@gmail.com

data

In [3]:
''' Колонки '''

ticker = 'ATEX'

# tickers_list = ['AXP', 'INTC']
# 


tickers_list = ['FOCS', 'TRUP', 'HLNE', 'NMIH', 'EVER', 'LC', 'TFC', 'MTB', 'EQH', 'FCNCA', 'DFS', 'WBS', 'RY', 'BKU', 'MKL', 'ERIE', 'STT', 'TCBI', 'OMF', 'LPLA', 'RRBI', 'AIZ', 'IBN', 'ESGR', 'GL', 'WRB', 'CNS', 'MQ', 'APAM', 'JPM', 'MTG', 'RJF', 'CME'] #42



required_year = '2022'

''' время '''
stime='2015-01-01'
period = '1mo'  # '1d' '1wk'

In [4]:
''' добавляем анализ процента ко всем колонкам'''
def change_percent_all_columns(df):
    x = df.columns.to_list()
    for i in range(len(x)):
        v = df[x[i]].to_list()
        l = []
        for y in range(len(v)):
            try:
                if y != len(v):
                    if v[y] < 0 and v[y+1] > 0:
                        l.append('Minus')
                    elif v[y] > 0 and v[y+1] < 0:
                        l.append('Plus')
                    else:
                        vv = (v[y] / (v[y+1] / 100)) - 100
                        l.append(vv) 
                else:
                    vv = 0
                    l.append(vv)
                    # print(vv)
            except:
                l.append(0)
        df[f'change % {x[i]}'] = l
    return df

In [5]:
''' добавляем цифру к году, чтобы видень данные как начало года, а не как факт '''
def change_index_plus_one(df):
    df_index = df.index.to_list()
    for i in range(len(df_index)):
        df_index[i] = str(int(df_index[i])+1)
    df['Date_index'] = df_index
    df = df.set_index('Date_index')
    return df

In [6]:
def one_stock_analyse(ticker, api_key, required_year):
    
    try:

        ''' запрос к yahoo за ценами '''
        def get_data_from_ticker(tick, start_time='2017-01-01', interval='1mo'):
            ticker = yf.Ticker(tick)

            df = ticker.history(start=start_time, end=None, interval=interval)
            x = pd.DataFrame(df)
            x.rename(columns={"Close": tick}, inplace=True)
            z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
            return z

        df = get_data_from_ticker(ticker)

        ''' получаем цену на период отчета '''
        income_statement = fa.income_statement(ticker, api_key)
        income_statement = income_statement.T
        income_statement = change_index_plus_one(income_statement)
        accepted_date = income_statement['acceptedDate'][0][:10]
        df_report = get_data_from_ticker(ticker, accepted_date)
        report_price = df_report.iloc[0,0]

        ''' получаем актуальную цену акции '''
        today = datetime.date.today()
        sdate = today - relativedelta(months=3)
        sdate = str(sdate)
        df_current_price = get_data_from_ticker(ticker, sdate, '1d')
        current_price = df_current_price.iloc[-1,0]

        ''' меняем индекс для корреляции с df ключевых метрик '''
        def change_index(df):
            df_index = df.index.to_list()
            for i in range(len(df_index)):
                t = df_index[i].date()
                df_index[i] = t.strftime('%Y-%m-%d')
            df['Date_index'] = df_index
            df = df.set_index('Date_index')
            return df

        df = change_index(df)


        def get_key_metricks(ticker, api_key, required_year):
            key_metrics = fa.key_metrics(ticker, api_key, period='annual')
            df_key_metr = key_metrics.T
            df_key_metr = change_index_plus_one(df_key_metr)
            df_key_metr = df_key_metr.loc[required_year:'2000'].copy(deep=True)
            return df_key_metr



        df_key_metr = get_key_metricks(ticker, api_key, required_year)


        if df_key_metr.index[0] == required_year and f'{int(required_year)+1}-01-01' in df.index.to_list():
            last_year = True
        else:
            last_year = False



        def get_index(df):
            index_years = df.index.to_list()
            return index_years

        index_years = get_index(df_key_metr)



        def get_price_from_df(df, list_dates):
            new_list = []
            for i in range(len(list_dates)):
                x = int(list_dates[i])
                x += 1
                x = str(x)
                try:
                    r = df.loc[f'{x}-01-01']
                    r = float(r)
                    new_list.append(r)
                except:
                    new_list.append(0)

            return new_list

        stock_year_prices = get_price_from_df(df, index_years)


        df_key_metr[ticker] = list(stock_year_prices)
        df_key_metr.drop('period', axis=1, inplace=True)

        def change_percent_all_columns(df):
            x = df.columns.to_list()
            for i in range(len(x)):
                v = df[x[i]].to_list()
                l = []
                for y in range(len(v)):
                    try:
                        if y != len(v):
                            vv = (v[y] / (v[y+1] / 100)) - 100
                            l.append(vv)
                        else:
                            vv = 0
                            l.append(vv)
                    except:
                        l.append(0)
                df[f'change % {x[i]}'] = l
            return df

        df_with_changes = change_percent_all_columns(df_key_metr)

        # if df_with_changes[f'change % {ticker}'][0] > df_with_changes[f'change % {ticker}'][1] and df_with_changes['change % debtToAssets'][0] < df_with_changes['change % debtToAssets'][1]:
        #     better_less_last = "Short"
        if df_with_changes[f'change % {ticker}'][0] < df_with_changes[f'change % {ticker}'][1] and df_with_changes['change % debtToAssets'][0] > df_with_changes['change % debtToAssets'][1] and df_with_changes['change % debtToAssets'][0] > df_with_changes[f'change % {ticker}'][0]:
            better_less_last = "Long"
        else:
            better_less_last = False



        change_price = current_price / (report_price/100) - 100


        if last_year == True and better_less_last != False:
            # df_last = df_with_changes.reindex(index=df_with_changes.index[::-1])
            r = [ticker, accepted_date, report_price, current_price, change_price, df_with_changes['change % debtToAssets'][0], better_less_last]
        else:
            r = None 
    except:
        r = None
        
    print(r)
    return r
    # return df_report
    # return accepted_date

results of stock's data

In [7]:
def get_results_da_price(tickers_list, api_key, required_year):
    l = []
    for i in range(len(tickers_list)):
        r = one_stock_analyse(tickers_list[i], api_key, required_year)
        if r == None:
            pass
        else:
            l.append(r)
    return l
  
df = pd.DataFrame(get_results_da_price(tickers_list, api_key, required_year), columns = ['Ticker', 'Accepted_Date', 'Price', 'Current Price', 'Change % price', 'change % debtToAssets', 'Side'])
df

['FOCS', '2022-02-17', 45.7400016784668, 50.15999984741211, 9.663310027874644, 5.9928614407672995, 'Long']
['TRUP', '2022-02-15', 89.12000274658203, 59.90999984741211, -32.7760345589646, 28.883976391367014, 'Long']
['HLNE', '2022-05-26', 66.01810455322266, 79.77999877929688, 20.845636691946552, -3.602026988452465, 'Long']
['NMIH', '2022-02-15', 20.6200008392334, 23.68000030517578, 14.839958008731813, -1.8554070311602402, 'Long']
None
['LC', '2023-02-09', 10.09000015258789, 10.09000015258789, 0.0, 35.191641229411545, 'Long']
['TFC', '2022-02-22', 54.317047119140625, 48.81999969482422, -10.120298719956224, 1.3091467824981606, 'Long']
None
['EQH', '2022-12-30', 32.06999969482422, 32.880001068115234, 2.5257292828154903, 1.7590307942997185, 'Long']
None
None
None
None
None
None
None
None
None
None
None
None
['AIZ', '2022-02-22', 179.4062042236328, 130.25, -27.399389244286553, 494.7450324339428, 'Long']
None
['ESGR', '2022-02-24', 261.1499938964844, 243.00999450683594, -6.946199430829338, 8.

,Ticker,Accepted_Date,Price,Current Price,Change % price,change % debtToAssets,Side
0,FOCS,2022-02-17,45.740002,50.160000,9.663310,5.992861,Long
1,TRUP,2022-02-15,89.120003,59.910000,-32.776035,28.883976,Long
2,HLNE,2022-05-26,66.018105,79.779999,20.845637,-3.602027,Long
3,NMIH,2022-02-15,20.620001,23.680000,14.839958,-1.855407,Long
4,LC,2023-02-09,10.090000,10.090000,0.000000,35.191641,Long
5,TFC,2022-02-22,54.317047,48.820000,-10.120299,1.309147,Long
6,EQH,2022-12-30,32.070000,32.880001,2.525729,1.759031,Long
7,AIZ,2022-02-22,179.406204,130.250000,-27.399389,494.745032,Long
8,ESGR,2022-02-24,261.149994,243.009995,-6.946199,8.833591,Long


results

In [8]:
l  = []
for i in range(len(df['Change % price'])):
    if df['Side'][i] == 'Long':
        r = df['Change % price'][i] - df['change % debtToAssets'][i]
        r = r *(-1) if r < 0 else r
        l.append(r)
    elif df['Side'][i] == 'Short':
        r = df['Change % price'][i] - df['change % debtToAssets'][i]
        r = r *(-1) if r > 0 else r

        l.append(r)
    else:
        l.append(0)

df['Differense'] = l

In [9]:
df

,Ticker,Accepted_Date,Price,Current Price,Change % price,change % debtToAssets,Side,Differense
0,FOCS,2022-02-17,45.740002,50.160000,9.663310,5.992861,Long,3.670449
1,TRUP,2022-02-15,89.120003,59.910000,-32.776035,28.883976,Long,61.660011
2,HLNE,2022-05-26,66.018105,79.779999,20.845637,-3.602027,Long,24.447664
3,NMIH,2022-02-15,20.620001,23.680000,14.839958,-1.855407,Long,16.695365
4,LC,2023-02-09,10.090000,10.090000,0.000000,35.191641,Long,35.191641
5,TFC,2022-02-22,54.317047,48.820000,-10.120299,1.309147,Long,11.429446
6,EQH,2022-12-30,32.070000,32.880001,2.525729,1.759031,Long,0.766698
7,AIZ,2022-02-22,179.406204,130.250000,-27.399389,494.745032,Long,522.144422
8,ESGR,2022-02-24,261.149994,243.009995,-6.946199,8.833591,Long,15.779791


In [10]:
df['Differense'].sum()

691.7854860714214

add_key_metrics

In [11]:

# def add_key_metrics(df):
#     v = 'revenuePerShare'
#     v1 = 'netIncomePerShare'
#     v2 = 'inventoryTurnover'
#     v3 = 'capexPerShare'
#     v4 = 'cashPerShare'
#     v5 = 'bookValuePerShare'
#     v6 = 'daysPayablesOutstanding'
#     v7 = 'receivablesTurnover'
#     l = []
#     l1 = []
#     l2 = []
#     l3 = []
#     l4 = []
#     l5 = []
#     l6 = []
#     l7 = []
#     for i in range(len(df['Ticker'])):
#         if i == 0:
#             key_metrics = fa.key_metrics(df['Ticker'][i], api_key, period='annual').T
#             # print(key_metrics)
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             l.append(key_metrics_change.iloc[0][f'change % {v}'])
#             l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
#             l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
#             l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
#             l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
#             l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
#             l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
#             l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
#         else:
#             key_metrics = fa.key_metrics(df['Ticker'][i], api_key, period='annual').T
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             l.append(key_metrics_change.iloc[0][f'change % {v}'])
#             l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
#             l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
#             l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
#             l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
#             l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
#             l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
#             l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
#     df[f'change % {v}'] = l
#     df[f'change % {v1}'] = l1
#     df[f'change % {v2}'] = l2
#     df[f'change % {v3}'] = l3
#     df[f'change % {v4}'] = l4
#     df[f'change % {v5}'] = l5
#     df[f'change % {v6}'] = l6
#     df[f'change % {v7}'] = l7
    
#     return df

# df_with_parameters = add_key_metrics(df)

In [12]:
'''
revenuePerShare < -10 = повышает шансы к лонгу
netIncomePerShare == Minus = повышает шансы к Лонгу
netIncomePerShare == Plus = повышает шансы к Шорту
bookValuePerShare > 100 = повышает шансы к Лонгу
capexPerShare > 100 = повышает шансы к Лонгу
daysPayablesOutstanding > 50 = повышает шансы к Лонгу
'''

'\nrevenuePerShare < -10 = повышает шансы к лонгу\nnetIncomePerShare == Minus = повышает шансы к Лонгу\nnetIncomePerShare == Plus = повышает шансы к Шорту\nbookValuePerShare > 100 = повышает шансы к Лонгу\ncapexPerShare > 100 = повышает шансы к Лонгу\ndaysPayablesOutstanding > 50 = повышает шансы к Лонгу\n'

add_income_statement

In [13]:
# def add_income_statement(df):
#     v = 'revenue'
#     v1 = 'costOfRevenue'
#     v2 = 'grossProfit'
#     v3 = 'researchAndDevelopmentExpenses'
#     v4 = 'costAndExpenses'
#     v5 = 'ebitda'
#     v6 = 'operatingIncome'
#     v7 = 'eps'
#     l = []
#     l1 = []
#     l2 = []
#     l3 = []
#     l4 = []
#     l5 = []
#     l6 = []
#     l7 = []
#     for i in range(len(df['Ticker'])):
#         if i == 0:
#             key_metrics = fa.income_statement(df['Ticker'][i], api_key, period='annual').T
#             # print(key_metrics)
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             l.append(key_metrics_change.iloc[0][f'change % {v}'])
#             l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
#             l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
#             l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
#             l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
#             l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
#             l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
#             l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
#         else:
#             key_metrics = fa.income_statement(df['Ticker'][i], api_key, period='annual').T
#             key_metrics_change = change_percent_all_columns(key_metrics)
#             l.append(key_metrics_change.iloc[0][f'change % {v}'])
#             l1.append(key_metrics_change.iloc[0][f'change % {v1}'])
#             l2.append(key_metrics_change.iloc[0][f'change % {v2}'])
#             l3.append(key_metrics_change.iloc[0][f'change % {v3}'])
#             l4.append(key_metrics_change.iloc[0][f'change % {v4}'])
#             l5.append(key_metrics_change.iloc[0][f'change % {v5}'])
#             l6.append(key_metrics_change.iloc[0][f'change % {v6}'])
#             l7.append(key_metrics_change.iloc[0][f'change % {v7}'])
#     df[f'change % {v}'] = l
#     df[f'change % {v1}'] = l1
#     df[f'change % {v2}'] = l2
#     df[f'change % {v3}'] = l3
#     df[f'change % {v4}'] = l4
#     df[f'change % {v5}'] = l5
#     df[f'change % {v6}'] = l6
#     df[f'change % {v7}'] = l7
    
#     return df

# df_with_parameters = add_income_statement(df)

In [14]:
# df_with_parameters